<a href="https://www.kaggle.com/code/yacharki/training-testing-the-amazon-model?scriptVersionId=191565891" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# Downloads
!pip install contractions
!pip install textsearch
!pip install tqdm

import nltk
nltk.download('punkt')

# Fundamental classes
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np

# Time
import time
import datetime

# Preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import contractions
from bs4 import BeautifulSoup
import re
import tqdm
import unicodedata

seed = 3541
np.random.seed(seed)

In [ ]:
# Define a dummy loss to bypass the error during model loading
def dummy_loss(y_true, y_pred):
    return tf.reduce_mean(y_pred - y_true)

# Loading the model Trained on Amazon reviews
modelAmazon = keras.models.load_model(
    '/kaggle/input/pre-trained-model-binary-cnn-nlp-amazon-reviews/tensorflow1/pre_trained_sentiment_analysis_cnn_model_amazon_reviews/1/Binary_Classification_86_Amazon_Reviews_CNN.h5',
    compile=False
)

# Compile the model with the correct loss function and reduction
modelAmazon.compile(
    optimizer='adam',
    loss=keras.losses.BinaryCrossentropy(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE),
    metrics=['accuracy']
)

# Loading Amazon test data
dataset_test_Amazon = pd.read_csv('/kaggle/input/amazon-reviews-for-sa-binary-negative-positive-csv/amazon_review_sa_binary_csv/test.csv')

# Loading Amazon train data (to be used on the label encoder)
dataset_train_Amazon = pd.read_csv('/kaggle/input/amazon-reviews-for-sa-binary-negative-positive-csv/amazon_review_sa_binary_csv/train.csv')

# Shuffling the Test Data
test_Amazon = dataset_test_Amazon.sample(frac=1)
train_Amazon = dataset_train_Amazon.sample(frac=1)

# Taking a tiny portion of the database (because it will only be used on the label encoder)
train_Amazon = dataset_train_Amazon.iloc[:100, :]

# Taking only necessary columns
y_test_Amazon = test_Amazon['class_index'].values
X_train_Amazon = train_Amazon['review_text'].values
y_train_Amazon = train_Amazon['class_index'].values

# Preprocess corpus function
def pre_process_corpus(corpus):
    processed_corpus = []
    for doc in tqdm.tqdm(corpus):
        doc = contractions.fix(doc)
        doc = BeautifulSoup(doc, "html.parser").get_text()
        doc = unicodedata.normalize('NFKD', doc).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        doc = re.sub(r'[^a-zA-Z\s]', '', doc, re.I|re.A)
        doc = doc.lower()
        doc = doc.strip()
        processed_corpus.append(doc)
    return processed_corpus

# Preprocessing the Data
X_test_Amazon = pre_process_corpus(test_Amazon['review_text'].values)
X_train_Amazon = pre_process_corpus(X_train_Amazon)

# Creating and Fitting the Tokenizer
t = Tokenizer(oov_token='<UNK>')
t.fit_on_texts(X_train_Amazon)
t.word_index['<PAD>'] = 0

# Transforming text to sequences 
X_test_Amazon = t.texts_to_sequences(X_test_Amazon)
X_train_Amazon = t.texts_to_sequences(X_train_Amazon)

# Padding the transformed text (sentences) to maximum length of 220
X_test_Amazon = sequence.pad_sequences(X_test_Amazon, maxlen=220)
X_train_Amazon = sequence.pad_sequences(X_train_Amazon, maxlen=220)

# Creating and Fitting the label encoder
le = LabelEncoder()
num_classes = 2  # positive -> 1, negative -> 0
y_train_Amazon = le.fit_transform(y_train_Amazon)

# Transforming the labels
y_test_Amazon = le.transform(y_test_Amazon)

In [ ]:
# Evaluating the models

#Testing Amazon Classifier on Amazon Test Data
print(" Testing Amazon Classifier on Amazon Test Data")
scores = modelAmazon.evaluate(X_test_Amazon, y_test_Amazon, verbose=1)
print("Accuracy: %.2f%% /n" % (scores[1]*100))